In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geoviews as gv
import panel as pn
import hvplot.xarray

In [ ]:
gv.extension("bokeh")

## Generate some sample data to play with

In [ ]:
lon = np.arange(-180, 180, 2)
lat = np.arange(-80, 80, 2)
lonG, latG = np.meshgrid(lon, lat)
lonG, latG

In [ ]:
phase = np.arange(-np.pi, np.pi + np.pi/10, np.pi/10)
phase

In [ ]:
def cosinewave(x, phase=0, scale=360):
    return np.cos(2*np.pi*x/scale + phase)

data = np.stack([cosinewave(lonG, ph) for ph in phase])
data.shape

## Store sample data in xarray Dataset, `ds`

In [ ]:
# ds = xr.Dataset(
#     data_vars=dict(data=(("phase", "x", "y"), data)),
#     coords=dict(lon=(("x", "y"), lonG), lat=(("x", "y"), latG), phase=phase),
# )
ds = xr.Dataset(
    data_vars=dict(data=(("phase", "lat", "lon"), data)),
    coords=dict(lon=(("lon"), lon), lat=(("lat"), lat), phase=phase),
)
ds

In [ ]:
xr.plot.pcolormesh(ds["data"].isel(phase=10), "lon", "lat");

## Now try some stuff with GeoViews

In [ ]:
gv.Dataset(ds["data"]).to(gv.FilledContours, groupby="phase")

<https://geoviews.org/gallery/bokeh/filled_contours.html#bokeh-gallery-filled-contours>

In [ ]:
gv.FilledContours((ds.lon, ds.lat, ds.data[0, ...]))

https://holoviews.org/reference/containers/bokeh/DynamicMap.html  
http://holoviews.org/reference/containers/bokeh/HoloMap.html  
https://panel.holoviz.org/user_guide/Widgets.html  
`pn.widgets.DiscreteSlider(name="Phase", options=list(ds.phase.values), value=0)`

HoloMap takes longer to generate upfront

In [ ]:
def contours(phase=0):
    return gv.FilledContours((ds.lon, ds.lat, ds.sel(phase=phase).data))

hmap = gv.HoloMap({ph:contours(ph) for ph in ds.phase.values}, kdims=["Phase"])
hmap

DynamicMap might be smarter?

In [ ]:
dmap = gv.DynamicMap(
    lambda ph: gv.FilledContours((ds.lon, ds.lat, ds.sel(phase=ph).data)).opts(cmap="RdBu"),
    kdims=[gv.Dimension("phase", values=ds.phase.values, default=0)]
)
dmap

In [ ]:
dmap2 = (
    dmap
    * gv.feature.coastline()
).opts(projection=ccrs.Mollweide(), aspect=2)
dmap2

## Wrap with Panel to make it servable

In [ ]:
pn.Pane(dmap2).servable()